In [6]:
!pip install skforecast
!pip install dask[dataframe]
!pip install skforecast
!pip install dask[dataframe]
!pip install lightgbm
!pip install matplotlib
!pip install xgboost

sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x1f44d4c3640>
C:\Users\34634\anaconda3\lib\site-packages\IPython\utils\_process_win32.py:145: ResourceWarning: unclosed file <_io.BufferedWriter name=4>
  return process_handler(cmd, _system_body)
C:\Users\34634\anaconda3\lib\site-packages\IPython\utils\_process_win32.py:145: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return process_handler(cmd, _system_body)
C:\Users\34634\anaconda3\lib\site-packages\IPython\utils\_process_win32.py:145: ResourceWarning: unclosed file <_io.BufferedReader name=6>
  return process_handler(cmd, _system_body)


sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x1f44dcff280>


sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x1f44dcff460>


sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x1f44dcff2e0>


sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x1f44dcff520>


sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x1f44dcff4c0>


In [7]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import joblib
import matplotlib.pyplot as plt
import warnings
import csv

plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
plt.rcParams['font.size'] = 10

from sklearn.metrics import mean_squared_error
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

warnings.filterwarnings('once')

In [8]:
steps = [5,10,15,20,25,30]
lags = [2,5,7,10]

In [9]:
old_data = pd.read_csv('Fitabase Data 4.12.16-5.12.16/test_train_data.csv')
data = pd.read_csv('FitBit API Database/test_train_data_api_merged.csv')

print("Archivo : test_train_data.csv")
print(old_data)
print("\n")
print("Archivo : test_train_data_api_merged.csv")
print(data)

Archivo : test_train_data.csv
                Id                 Time   HeartRate  Intensity  Calories
0       2022484408  2016-04-12 07:21:00  101.600000          1   3.32064
1       2022484408  2016-04-12 07:22:00   87.888889          1   3.94326
2       2022484408  2016-04-12 07:23:00   58.000000          0   1.34901
3       2022484408  2016-04-12 07:24:00   58.000000          0   1.03770
4       2022484408  2016-04-12 07:25:00   56.777778          0   1.03770
...            ...                  ...         ...        ...       ...
333141  8877689391  2016-05-12 13:55:00   60.666667          0   1.33353
333142  8877689391  2016-05-12 13:56:00   61.875000          0   1.33353
333143  8877689391  2016-05-12 13:57:00   58.142857          0   1.33353
333144  8877689391  2016-05-12 13:58:00   61.200000          0   1.33353
333145  8877689391  2016-05-12 13:59:00   58.000000          0   1.33353

[333146 rows x 5 columns]


Archivo : test_train_data_api_merged.csv
           Id           

In [11]:
modelos = [
    ("LightGBM", lgb.LGBMRegressor(objective='regression', random_state=123)),
    ("XGBoost", XGBRegressor(random_state=123)),
    ("KNN", KNeighborsRegressor(n_neighbors=5)),
    ("RandomForest", RandomForestRegressor(n_estimators=100, max_depth=None, random_state=123)),
    ("MLP",MLPRegressor(random_state=123, max_iter=2000))
]


In [ ]:
resultados = {}

for nombre_modelo, modelo in modelos:
    for step in steps:  
        datos_train = data.iloc[:-step]  
        datos_test  = data.iloc[-step:] 
        datos_train = datos_train.bfill();
        datos_test = datos_test.bfill();

        for lag in lags:
            forecaster = ForecasterAutoreg(
                regressor=modelo,
                lags=lag
            )
            forecaster.fit(y=datos_train['HeartRate'], exog=datos_train[['Calories', 'Steps', 'Distance']])
            predicciones = forecaster.predict(steps=step, exog=datos_test[['Calories', 'Steps', 'Distance']])
            error_mse = mean_squared_error(y_true=datos_test['HeartRate'], y_pred=predicciones)
            resultados[(nombre_modelo, step, lag)] = error_mse
            



nombre_archivo = "resultados_modelos.csv"

# Abrir el archivo CSV en modo de escritura
with open(nombre_archivo, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Escribir el encabezado del archivo CSV
    writer.writerow(['Modelo', 'Steps', 'Lags', 'MSE'])

    # Escribir los resultados en el archivo CSV
    for nombre_modelo, _, _ in modelos:
        for step in steps:
            for lag in lags:
                error_mse = resultados[(nombre_modelo, step, lag)]
                # Escribir una fila en el archivo CSV
                writer.writerow([nombre_modelo, step, lag, error_mse])

print(f"Los resultados se han guardado en el archivo '{nombre_archivo}'.")


C:\Users\34634\anaconda3\lib\site-packages\joblib\externals\loky\backend\context.py:150: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] El sistema no puede encontrar el archivo especificado
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\34634\anaconda3\lib\site-packages\joblib\externals\loky\backend\context.py", line 227, in _count_physical_cores
    cpu_info = subprocess.run(
  File "C:\Users\34634\anaconda3\lib\subprocess.py", line 505, in run
    with Popen(*popenargs, **kwargs) as process:
  File "C:\Users\34634\anaconda3\lib\subprocess.py", line 951, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\34634\anaconda3\lib\subprocess.py", line 1420, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000620 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 591
[LightGBM] [Info] Number of data points in the train set: 20287, number of used features: 5
[LightGBM] [Info] Start training from score 65.931680
mse: 18.29665713319097
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000698 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 882
[LightGBM] [Info] Number of data points in the train set: 20284, number of used features: 8
[LightGBM] [Info] Start training from score 65.932656
mse: 12.270804839130614
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000794 seconds.
You can set `force_col_wise=true` to re

mse: 5.416750821478494
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001595 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1076
[LightGBM] [Info] Number of data points in the train set: 20257, number of used features: 10
[LightGBM] [Info] Start training from score 65.948462
mse: 5.238815725696202
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002955 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1367
[LightGBM] [Info] Number of data points in the train set: 20254, number of used features: 13
[LightGBM] [Info] Start training from score 65.947763
mse: 5.099608377071204
mse: 9.336941761095659
mse: 9.69849955443642
mse: 10.174817495018942
mse: 9.263180770137115
mse: 3.9474637668245123
mse: 3.7480691558288526
mse: 3.733396573900245
mse: 3.9573020189403905
mse: 4.394836894296653
mse: 4.52195168856706
mse: 4.582240

C:\Users\34634\anaconda3\lib\site-packages\pandas\core\algorithms.py:1814: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  return lib.map_infer(values, mapper, convert=convert)
C:\Users\34634\anaconda3\lib\site-packages\pandas\core\algorithms.py:1814: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  return lib.map_infer(values, mapper, convert=convert)


mse: 17.624000000000002


...

C:\Users\34634\anaconda3\lib\site-packages\pandas\core\algorithms.py:1814: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  return lib.map_infer(values, mapper, convert=convert)
C:\Users\34634\anaconda3\lib\site-packages\pandas\core\algorithms.py:1814: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  return lib.map_infer(values, mapper, convert=convert)


mse: 6.567686250000003


C:\Users\34634\anaconda3\lib\site-packages\pandas\core\algorithms.py:1814: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  return lib.map_infer(values, mapper, convert=convert)
C:\Users\34634\anaconda3\lib\site-packages\pandas\core\algorithms.py:1814: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  return lib.map_infer(values, mapper, convert=convert)


mse: 12.95196019758713


C:\Users\34634\anaconda3\lib\site-packages\pandas\core\algorithms.py:1814: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  return lib.map_infer(values, mapper, convert=convert)
